# Intro

* Loading Data and Libraries

In [ ]:
# loading libraries

import numpy as np
import pandas as pd

In [ ]:
# reading dataset

df = pd.read_csv("products.csv")
print("the dataset has " + str(df.shape[0]) + " observations (ice cream flavors) and " + str(df.shape[1]) + " features " + str([col for col in df.columns]))

# dataset: https://www.kaggle.com/datasets/tysonpo/ice-cream-dataset

# Inspiration Notebooks:
# Using Ice Cream Ingredients to Predict Rating: https://www.kaggle.com/code/gcdatkin/using-ice-cream-ingredients-to-predict-rating
# Finding the Best Ice Cream: https://www.kaggle.com/code/kelvintran1998/finding-the-best-ice-cream
# EDA ideas: McDonalds Ice Cream Machines Breaking: https://www.kaggle.com/code/aashidutt3/eda-mcdonalds-ice-cream-machines-breaking

In [ ]:
df.head(10)

* Data Cleaning: transform brand feature to more readable names

In [ ]:
# change brand name to be more readable

df['brand'] = df['brand'].replace(['hd','breyers','bj','talenti'],['HaagenDazs','Breyers','BenJerrys','Talenti'])
df['brand'].value_counts()

# Exploratory Data Analysis

* Exploratory Data Analysis: Color Scheme

In [ ]:
# loading dataviz libraries

import seaborn as sns
import matplotlib.pyplot as plt
from pylab import rcParams

sns.set_theme(style="whitegrid")
sns.despine(left=True, bottom=True)

In [ ]:
# color scheme

class clr:
    S = '\033[1m' + '\033[96m' # chance color text
    E = '\033[0m'
    
my_colors = ['#1F6082', '#6A2340', '#83563f', '#495057']

sns.palplot(sns.color_palette(my_colors))

print(clr.S + "Notebook Color Scheme: " + clr.E)
print(r"Ben Jerry's, Häagen-Dazs, Talenti, Breyers")


In [ ]:
# create color reference dict
# the idea is to use this dict to sort the color code in the same way of the plot
# colors = ['#1F6082', '#6A2340', '#83563f', '#171717']

color_reference = {'BenJerrys':'#1F6082', 'Breyers':'#495057', 'HaagenDazs':'#6A2340', 'Talenti':'#83563f'}
color_reference

In [ ]:
# setting and standardizing parameters for all graphs

sns.set()
plt.style.use('seaborn-v0_8-notebook')
%matplotlib inline
rcParams['figure.figsize'] = 12,8
rcParams['figure.titlesize'] = 14
rcParams['font.size'] = 15

* Exploratory Data Analysis

In [ ]:
# change column names

flavors = df['brand'].value_counts().reset_index().sort_values('brand', ascending=False)
flavors = flavors.rename(columns = {'index':'brand', 'brand':'value_counts'})
flavors

In [ ]:
# plot the brands flavors counts

plt.figure(figsize=(6, 4))
colors = flavors['brand'].replace(color_reference) # replace elements in a list using dictionary lookup
colors = colors.tolist() # list of colors for palette parameter in sns plot
splot = sns.barplot(data=flavors, x='brand', y='value_counts', palette=colors)
plt.ylim([0,80])
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.0f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.xlabel('Brand', fontsize=14)
plt.ylabel('# Flavors', fontsize=14)


* Exploratory Data Analysis: Flavors vs Rating

In [ ]:
# rating distribution

plt.figure(figsize=(6, 4))
sns.histplot(data=df, x='rating', hue='brand', multiple='stack', palette=color_reference)
plt.xlim([0,5.0])
plt.xticks(np.arange(0,5.1, step=0.5))
plt.xlabel('Average Rating', fontsize=14)
plt.ylabel('Flavors', fontsize=14)

Calculating kurtosis and Skewness

In [ ]:
# import scipy
from scipy.stats import skew, kurtosis

In [ ]:
# calculate the skewness

skewness = skew(df['rating'], axis=0)

if skewness == 0:
    print('the skewness value is ' + str(round(skewness,2)) + '. Then normally distributed.')
if skewness > 0:
    print('the skewness value is ' + str(round(skewness,2)) + '. Then more weight in the left tail of the distribution.')
if skewness < 0:
    print('the skewness value is ' + str(round(skewness,2)) + '. Then more weight in the right tail of the distribution.')

In [ ]:
# calculate the kurtosis
kurt =  kurtosis(df['rating'], axis=0)

if kurt == 3:
    print('the skewness value is ' + str(round(kurt,2)) + '. Then the distribution is mesokurtic.')
if kurt < 3:
    print('the skewness value is ' + str(round(kurt,2)) + '. Then the distribution is platykurtic (short tails).')
if kurt > 3:
    print('the skewness value is ' + str(round(kurt,2)) + '. Then the distribution is leptokurtic (thinner in the center and fatter tails. ie more outliers)')

In [ ]:
# change column names

ratings_by_brand = df.groupby('brand')['rating_count'].sum().reset_index().sort_values('brand', ascending=True)
ratings_by_brand

In [ ]:
# plot the brands flavors counts

plt.figure(figsize=(6, 4))
colors = ratings_by_brand['brand'].replace(color_reference) # replace elements in a list using dictionary lookup
colors = colors.tolist() # list of colors for palette parameter in sns plot
splot = sns.barplot(data=ratings_by_brand, x='brand', y='rating_count', palette=colors)
plt.ylim([0,9000])
for p in splot.patches:
    splot.annotate(format(p.get_height(), '.0f'), 
                   (p.get_x() + p.get_width() / 2., p.get_height()), 
                   ha = 'center', va = 'center', 
                   xytext = (0, 9), 
                   textcoords = 'offset points')
plt.xlabel('Brand', size=14)
plt.ylabel('# Reviews', size=14)

In [ ]:
# scatter plot by brand

sns.relplot(data=df,
                x='rating', y='rating_count',
                hue='brand',
                col='brand',
                palette=my_colors,
                sizes=(1, 6), linewidth=0, col_wrap=2
                )

In [ ]:
# scatter plot by rating vs. brand

_, axs = plt.subplots(nrows=1, ncols=2) # making a subplot with 1 row and 2 columns
sns.stripplot(data=df, x='brand', y='rating', palette=my_colors, ax=axs[0])
sns.boxplot(data=df, x='brand', y='rating', palette=my_colors, ax=axs[1])

* Flavors with low rating counts

In [ ]:
low_rating_count = df.query('rating_count < 10') # number choosed arbritrary
low_rating_count

* Worst Ice Creams and checking their reviews

In [ ]:
worst10 = df.sort_values('rating').head(10)
worst10

In [ ]:
# calling the ice cream reviews dataset

df_reviews = pd.read_csv("reviews.csv")

In [ ]:
# create a list of values to filter by ice cream's keys

values_list = worst10['key'].unique()

# filtering by two conditions: ice creams worst ratings and one star reviews
filtered_reviews = df_reviews[df_reviews['key'].isin(values_list)]
worst10_1star_reviews =  filtered_reviews.query('stars == 1') # getting one starred reviews for the 

# cleaning the worst10_1star_reviews df to become more readable
worst10_1star_reviews['brand'] = worst10_1star_reviews['brand'].replace(['hd','breyers','bj','talenti'],['HaagenDazs','Breyers','BenJerrys','Talenti'])
worst10_1star_reviews = worst10_1star_reviews.drop(columns=['key','author', 'date', 'taste', 'ingredients', 'texture', 'likes', 'helpful_yes', 'helpful_no'])
worst10_1star_reviews

* Data Cleaning: dropping unused columns

In [ ]:
# for this work, we don't need to use these columns, so we will drop it

df = df.drop(columns=['key','subhead','description'])
worst10 = worst10.drop(columns=['key','subhead','ingredients'])

* Top Ice Cream for each brand

In [ ]:
# top 10 Ben & Jerry's ice creams

top10_bj = df[df['brand'] == 'BenJerrys'].sort_values(by=['rating','rating_count'], ascending=False).head(10)
top10_bj


In [ ]:
# top 10 Haagen Dazs ice creams

top10_hd = df[df['brand'] == 'HaagenDazs'].sort_values(by=['rating','rating_count'], ascending=False).head(10)
top10_hd

In [ ]:
# top 10 Talenti ice creams

top10_tl = df[df['brand'] == 'Talenti'].sort_values(by=['rating','rating_count'], ascending=False).head(10)
top10_tl

In [ ]:
# top 10 Breyers ice creams

top10_br = df[df['brand'] == 'Breyers'].sort_values(by=['rating','rating_count'], ascending=False).head(10)
top10_br

* Bottom Ice Cream for each brand

In [ ]:
# bottom 10 Ben & Jerry's ice creams

bot10_bj = df[df['brand'] == 'BenJerrys'].sort_values(by=['rating','rating_count'], ascending=False).tail(10)
bot10_bj

In [ ]:
# bottom 10 Haagen Dazs ice creams

bot10_hd = df[df['brand'] == 'HaagenDazs'].sort_values(by=['rating','rating_count'], ascending=False).tail(10)
bot10_hd

In [ ]:
# bottom 10 Talenti ice creams

bot10_tl = df[df['brand'] == 'Talenti'].sort_values(by=['rating','rating_count'], ascending=False).tail(10)
bot10_tl

In [ ]:
# bottom 10 Breyers ice creams

bot10_br = df[df['brand'] == 'Breyers'].sort_values(by=['rating','rating_count'], ascending=False).tail(10)
bot10_br

* Top10 and Bot10 Comparison Table

In [ ]:
# creating a table to compare the average score with the average top10 score for each brand

grouped = df.groupby('brand')['rating'].mean().round(2)
new_column_rating_count = df.groupby('brand')['rating_count'].sum()
grouped = pd.concat([grouped, new_column_rating_count], axis=1).rename(columns={'rating':'rating_avg'})
grouped

In [ ]:
# calculating and inserting the top10 values to the table:
top10grouped = grouped.copy()
brand_dict = {'BenJerrys': top10_bj, 'Breyers': top10_br, 'HaagenDazs': top10_hd, 'Talenti': top10_tl}

for brand, top10_df in brand_dict.items():
    grouped_mean = top10_df.groupby('brand')['rating'].mean()
    
    grouped_count = top10_df.groupby('brand')['rating_count'].sum()
    grouped_top10 = pd.concat([grouped_mean, grouped_count], axis=1)
    grouped_top10 = grouped_top10.rename(columns={'rating': 'top10_rating_avg', 'rating_count':'top10_count'})

    # inserting these values into the table generated on the previous step:
    top10grouped.loc[brand, 'top10_rating_avg'] = grouped_top10['top10_rating_avg'][0]
    top10grouped.loc[brand, 'top10_count'] = grouped_top10['top10_count'][0]

top10grouped['top10_count'] = top10grouped['top10_count'].astype(int)
top10grouped

In [ ]:
grouped

* Bottom10 Comparison Table

In [ ]:
# calculating and inserting the bot10 values to the table:
bot10grouped = grouped.copy()
brand_dict = {'BenJerrys': bot10_bj, 'Breyers': bot10_br, 'HaagenDazs': bot10_hd, 'Talenti': bot10_tl}

for brand, bot10_df in brand_dict.items():
    grouped_mean = bot10_df.groupby('brand')['rating'].mean()
    
    grouped_count = bot10_df.groupby('brand')['rating_count'].sum()
    grouped_bot10 = pd.concat([grouped_mean, grouped_count], axis=1)
    grouped_bot10 = grouped_bot10.rename(columns={'rating': 'bot10_rating_avg', 'rating_count':'bot10_count'})

    # inserting these values into the table generated on the previous step:
    bot10grouped.loc[brand, 'bot10_rating_avg'] = grouped_bot10['bot10_rating_avg'][0]
    bot10grouped.loc[brand, 'bot10_count'] = grouped_bot10['bot10_count'][0]

bot10grouped['bot10_count'] = bot10grouped['bot10_count'].astype(int)
bot10grouped

# Saving the Results in csv Files

* Creating gist csv files for the medium articles

In [ ]:
#df.head().to_csv('icecream_head.csv', index=False, header=True, encoding='cp1252')
#df.tail().to_csv('icecream_tail.csv', index=False, header=True, encoding='cp1252')
#low_rating_count.to_csv('icecream_low_rating_count.csv', index=False, header=True, encoding='cp1252')
#worst10.to_csv('icecream_worst_flavors.csv', index=False, header=True, encoding='cp1252')
#worst10_1star_reviews.to_csv('icecream_worst10_1star_reviews.csv', index=False, header=True, encoding='cp1252')
#negative_reviews.to_csv('icecream_worst10_negative_reviews.csv', index=False, header=True, encoding='utf-8')
#top10grouped.to_csv('icecream_top10bot10grouped.csv', index=True, header=True, encoding='utf-8')


In [ ]:
# iterating the top and bottom tables to generate csv gists

#top_bot_list = {'top10_bj':top10_bj, 'top10_hd':top10_hd, 'top10_tl':top10_tl, 'top10_br':top10_br, 'bot10_bj':bot10_bj, 'bot10_hd':bot10_hd, 'bot10_tl':bot10_tl, 'bot10_br':bot10_br}

#for top_bot_csvname, top_bot_df  in top_bot_list.items():
    #csv_name = 'icecream_' + top_bot_csvname + '.csv'
    #top_bot_df.to_csv(csv_name, index=False, header=True, encoding='cp1252')

* Saving the pre-processing results

In [ ]:
# we will use this csv file in the next project phase

df.to_csv('pre-processed.csv', index=False, header=True, encoding='utf-8')